In [1]:
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import pymongo
from time import sleep as sleep
import random

In [60]:
# set up pymongo connection

# https://docs.mongodb.com/manual/reference/default-mongodb-port/

conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# declare database
db = client.lol_information

# declare the collection
sr_items = db.sr_items
test_items = db.test_items
champions = db.champions

In [3]:
url ='https://leagueoflegends.fandom.com/wiki/Gold_efficiency_(League_of_Legends)'
pd.read_html(url)

[        Champion Statistic          Base Item  Item Cost Stat Raised  \
 0            Attack damage         Long Sword        350       10 AD   
 1            Ability power    Amplifying Tome        435       20 AP   
 2                    Armor        Cloth Armor        300    15 Armor   
 3         Magic resistance  Null-Magic Mantle        450       25 MR   
 4                   Health       Ruby Crystal        400      150 HP   
 5                     Mana   Sapphire Crystal        350      250 MP   
 6      Health regeneration  Rejuvenation Bead        150     50% HP5   
 7        Mana regeneration       Faerie Charm        250     50% MP5   
 8   Critical strike chance   Cloak of Agility        600    15% Crit   
 9             Attack speed             Dagger        300      12% AS   
 10     Flat movement speed              Boots        300       25 MS   
 
     Gold / Stat Bonus  
 0               35.00  
 1               21.75  
 2               20.00  
 3               18.00

In [4]:
item_eff_table = pd.read_html(url)[12]

In [5]:
item_eff_table

,Item,Cost,Amount,Availability
0,Abyssal Mask,2700,74.32%,All maps
1,Aegis of the Legion,1400,100.48%,All maps
2,Aether Wisp,850,76.76%,All maps
3,Amplifying Tome,435,100%,All maps
4,Anathema's Chains,2500,90.67%,All maps
...,...,...,...,...
210,Wyrmfallen Sacrifice,3400,140.44%,All maps
211,Youmuu's Ghostblade,3000,96.37%,All maps
212,Zeal,1050,100%,All maps
213,Zeke's Convergence,2400,85.42%,All maps


In [6]:
item_eff_table = item_eff_table.drop(columns=['Availability'])

In [7]:
item_old = item_eff_table['Item'].to_list()
item_new = []

amount_old = item_eff_table['Amount'].to_list()
amount_new = []

for i in range(len(item_old)):
    old_value = item_old[i]
    new_item_name = old_value.replace(" ","_")

    item_new.append(new_item_name)
    
    
    old_value = amount_old[i]
    new_value = old_value.split('%')[0]
    amount_new.append(new_value)

    
item_eff_table['Item'] = item_new

item_eff_table['Amount'] = amount_new

In [8]:
item_eff_table.head(50)

,Item,Cost,Amount
0,Abyssal_Mask,2700,74.32
1,Aegis_of_the_Legion,1400,100.48
2,Aether_Wisp,850,76.76
3,Amplifying_Tome,435,100
4,Anathema's_Chains,2500,90.67
5,Archangel's_Staff,2600,97.63
6,Ardent_Censer,2300,102.39
7,Axiom_Arc,3000,96.87
8,B._F._Sword,1300,107.69
9,Bami's_Cinder,1100,72.73


In [9]:
test_list = []
entry = {}

In [10]:
base_url='https://leagueoflegends.fandom.com/wiki/'

item_name = item_eff_table['Item'][0]
item_url = f'{base_url}{item_name}'

entry[f'{item_name}'] = {
'item_name':item_eff_table['Item'][0],
'cost':item_eff_table['Cost'][0],
'gold_efficiency_prcnt':item_eff_table['Amount'][0],
'item_url':item_url
}



test_list.append(entry)

In [11]:
test_list

[{'Abyssal_Mask': {'item_name': 'Abyssal_Mask',
   'cost': 2700,
   'gold_efficiency_prcnt': '74.32',
   'item_url': 'https://leagueoflegends.fandom.com/wiki/Abyssal_Mask'}}]

### - set up soup object(s)
### - parse through html to find/extract values - debate if using pandas scrape or bs4
### - set values to keys as needed
### - make master list of dictionaries with information
### - save to CSV and JSON

CONSIDERATIONS:
- You may need to edit the strings of items and their stats
- set up if statemetns to capture information -> if attk, then stat_values['attk']=stat

In [81]:
def item_finder():
# using beautiful soup this parses through a list taking the text for items in the soup object that are
# below 30 characters.
# we want to search for stats until we find the item description on the item page
 # https://leagueoflegends.fandom.com/wiki/Aegis_of_the_Legion
# everything above the desc. we keep, all below is ignored
# we return the dirty list of stats and their values as strings

# return example
# ['+10 ability haste', '+30 armor', '+30 magic resistance']



    item_entry_list = []
    count = 0
    for item in item_stats:
        item_stat = item_stats[count].get_text()
        if len(item_stat)<30:
            item_entry_list.append(item_stat)
        else:continue
        count+=1
    return item_entry_list

# --------------------------------------------------------------------------------------------------------------------#

def stat_cleaner(item_entry_list):
# takes a list of elements pulled from splinter, soup scrape and
# FOR EACH
# removes character, converts string numbers to INT
# rejoins words defining the stat into one EX: stat value  -> stat_value
# assembles stat values into a dictionary -> {stat_name: stat_value, stat_name: stat_value, stat_name: stat_value}

# returns the finished dictonary
# EXAMPLE:
# {'ability_haste': 10, 'armor': 30, 'magic_resistance': 30}

    stats_dict = {}
    for i in range(len(item_entry_list)):

        list_ele = item_entry_list[i]
        list_ele = list_ele.replace('+','')
        list_ele = list_ele.replace('%','')
        list_ele = list_ele.replace('.','')
        list_ele = list_ele.replace("'","")

        attribute = list_ele.split()
        
        

        attr_len = len(attribute)

        try:
            stat_value = int(attribute[0])

            if attr_len ==5:
                stat_name = '_'.join([attribute[i] for i in [1,2,3,4]])
                stats_dict[f'{stat_name}']=stat_value

            elif attr_len ==4:
                stat_name = '_'.join([attribute[i] for i in [1,2,3]])
                stats_dict[f'{stat_name}']=stat_value

            elif attr_len ==3:
                stat_name = '_'.join([attribute[i] for i in [1,2]])
                stats_dict[f'{stat_name}']=stat_value

            elif attr_len ==2:
                stat_name = '_'.join([attribute[i] for i in [1]])
                stats_dict[f'{stat_name}']=stat_value
            else:
                stat_name = '_'.join([attribute[i] for i in [1]])
                stats_dict[f'{stat_name}']=stat_value
        except ValueError:
            x=1
            continue
    return stats_dict

# --------------------------------------------------------------------------------------------------------------------#

def string_cleaner(item_name):
    clean_name = item_name
    clean_name = clean_name.replace('+','')
    clean_name = clean_name.replace('%','')
    clean_name = clean_name.replace('.','')
    clean_name = clean_name.replace("'","")
    return clean_name
# --------------------------------------------------------------------------------------------------------------------#

# def stat_sanitizer(stat_dict):
#     # search through dictionary made with stat stripper, return
    
    
    
#     clean_stat_dict = {}
#     key_list = ['attack_damage', 'health', 'ability_power', 'movement_speed', 'attack_speed', 'omnivamp', 'ability_haste', 'armor', 'magic_resistance']
#     for i in range(len(key_list)):
#         key = key_list[i]
#         try:
#             if stat_dict[key]:
#                 print()
#                 print(f'found {key}')
#                 sample_dict[f'{key}'] = stat_dict[key]

#             else:
#                 print()
#                 print('if statement was false')

#         except KeyError:
#             x=1
# #             print()
# #             print(f'{key} not found')
#             continue
#     return clean_stat_dict


In [13]:
# set up splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST chromedriver version for 97.0.4692 google-chrome
Driver [C:\Users\Alec\.wdm\drivers\chromedriver\win32\97.0.4692.71\chromedriver.exe] found in cache


#### secondary means of locatin item information, less reliable.
collapse_list_ele = soup_object.find('div',class_='mw-collapsible-content')
coll_list = collapse_list_ele.find_all('li')

split the string from each element in list to break it into entry[f'{stat}'] = value
stop when the length of the string is greater than 30 characters
remove the + & % signs from all text

# TEST BLOCK

In [65]:
test_master_list =[]
test_item_url_list = []
length = len(item_eff_table['Item'])

base_url='https://leagueoflegends.fandom.com/wiki/'

for i in range(0,1):
    i = 27
    
    # list and dictionaries -------------------------------------------------------------
    mongo_entry = {}
    entry = {}
    
    # item values -------------------------------------------------------------
    item_name = item_eff_table['Item'][i]
    item_url = f'{base_url}{item_name}'
    
    clean_name = (string_cleaner(item_name)).lower()
    
    browser.visit(item_url)
    html = browser.html
    soup_object = BeautifulSoup(html,'html.parser')
    item_stats = soup_object.find_all('div',class_='pi-data-value pi-font')

    item_entry_list = item_finder()
    stat_dict = stat_cleaner(item_entry_list)
    
    # aggregations -------------------------------------------------------------

    cost = int(item_eff_table['Cost'][i])
    gold_efficiency = float(item_eff_table['Amount'][i])

    effective_value = round((cost*gold_efficiency/100),2)
    
    # dictionary insertion -------------------------------------------------------------

    mongo_entry[f'{clean_name}'] = {
    'item_name':clean_name,
    'cost':cost,
    'gold_efficiency':gold_efficiency,
    'effective_value':effective_value,
    'stats':stat_dict,
    'item_url': item_url,

    }

    entry['item_name'] = clean_name
    entry['cost'] = cost
    entry['gold_efficiency'] = gold_efficiency
    entry['effective_value'] = effective_value
    entry['item_url'] = item_url

# ----------------------------------------------------------------------------------------------------------------------- #

    # combine scraped_dict with entry_dict.
    mongo_entry[f'{clean_name}'].update(stat_dict)

    entry.update(stat_dict)
    
    # insert mongo_entry into items collection
    test_items.insert_one(mongo_entry)
    
    # create lists of URL and list of dict_item_info
    test_item_url_list.append(item_url)
    test_master_list.append(entry)
#     wait_time = random.uniform(1.5,5)
#     sleep(wait_time)

# END TEST BLOCK

------------

# Summoners Rift Items

In [84]:
master_list =[]
mongo_list = []
item_url_list = []
length = len(item_eff_table['Item'])

base_url='https://leagueoflegends.fandom.com/wiki/'

for i in range(length):
    # list and dictionaries -------------------------------------------------------------
    mongo_entry = {}
    entry = {}
    
    # item values -------------------------------------------------------------
    item_name = item_eff_table['Item'][i]
    item_url = f'{base_url}{item_name}'
    
    clean_name = (string_cleaner(item_name)).lower()
    
    browser.visit(item_url)
    html = browser.html
    soup_object = BeautifulSoup(html,'html.parser')
    item_stats = soup_object.find_all('div',class_='pi-data-value pi-font')

    item_entry_list = item_finder()
    stat_dict = stat_cleaner(item_entry_list)
    
    # aggregations -------------------------------------------------------------

    cost = int(item_eff_table['Cost'][i])
    gold_efficiency = float(item_eff_table['Amount'][i])

    effective_value = round((cost*gold_efficiency/100),2)
    
    # dictionary insertion -------------------------------------------------------------

    mongo_entry[f'{clean_name}'] = {
    'item_name':clean_name,
    'cost':cost,
    'gold_efficiency':gold_efficiency,
    'effective_value':effective_value,
    'stats':stat_dict,
    'item_url': item_url,

    }

    entry['item_name'] = clean_name
    entry['cost'] = cost
    entry['gold_efficiency'] = gold_efficiency
    entry['effective_value'] = effective_value
    entry['item_url'] = item_url

# ----------------------------------------------------------------------------------------------------------------------- #

    # combine scraped_dict with entry_dict.
    mongo_entry[f'{clean_name}'].update(stat_dict)

    entry.update(stat_dict)
    
    # insert mongo_entry into items collection
    sr_items.insert_one(mongo_entry)
    
    # create lists of URL and list of dict_item_info
    mongo_list.append(mongo_entry)
    item_url_list.append(item_url)
    master_list.append(entry)
#     wait_time = random.uniform(1.5,5)
#     sleep(wait_time)

In [98]:
browser.quit()

In [85]:
master_list

[{'item_name': 'abyssal_mask',
  'cost': 2700,
  'gold_efficiency': 74.32,
  'effective_value': 2006.64,
  'item_url': 'https://leagueoflegends.fandom.com/wiki/Abyssal_Mask',
  'ability_haste': 10,
  'health': 450,
  'magic_resistance': 30},
 {'item_name': 'aegis_of_the_legion',
  'cost': 1400,
  'gold_efficiency': 100.48,
  'effective_value': 1406.72,
  'item_url': 'https://leagueoflegends.fandom.com/wiki/Aegis_of_the_Legion',
  'ability_haste': 10,
  'armor': 30,
  'magic_resistance': 30},
 {'item_name': 'aether_wisp',
  'cost': 850,
  'gold_efficiency': 76.76,
  'effective_value': 652.46,
  'item_url': 'https://leagueoflegends.fandom.com/wiki/Aether_Wisp',
  'ability_power': 30},
 {'item_name': 'amplifying_tome',
  'cost': 435,
  'gold_efficiency': 100.0,
  'effective_value': 435.0,
  'item_url': 'https://leagueoflegends.fandom.com/wiki/Amplifying_Tome',
  'ability_power': 20},
 {'item_name': 'anathemas_chains',
  'cost': 2500,
  'gold_efficiency': 90.67,
  'effective_value': 2266.7

In [86]:
len(master_list)

215

In [87]:
items_df = pd.DataFrame(master_list)
items_df = items_df.fillna(0)


In [88]:
items_df.head(50)

,item_name,cost,gold_efficiency,effective_value,item_url,ability_haste,health,magic_resistance,armor,ability_power,...,magic_penetration,critical_strike_chance,slow_resist,omnivamp,base_health_regeneration,health_on-hit,health_per_5_seconds,omnivamp_against_monsters,armor_penetration,tenacity
0,abyssal_mask,2700,74.32,2006.64,https://leagueoflegends.fandom.com/wiki/Abyssa...,10.0,450.0,30.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,aegis_of_the_legion,1400,100.48,1406.72,https://leagueoflegends.fandom.com/wiki/Aegis_...,10.0,0.0,30.0,30.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,aether_wisp,850,76.76,652.46,https://leagueoflegends.fandom.com/wiki/Aether...,0.0,0.0,0.0,0.0,30.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,amplifying_tome,435,100.00,435.00,https://leagueoflegends.fandom.com/wiki/Amplif...,0.0,0.0,0.0,0.0,20.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,anathemas_chains,2500,90.67,2266.75,https://leagueoflegends.fandom.com/wiki/Anathe...,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,archangels_staff,2600,97.63,2538.38,https://leagueoflegends.fandom.com/wiki/Archan...,0.0,200.0,0.0,0.0,60.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,ardent_censer,2300,102.39,2354.97,https://leagueoflegends.fandom.com/wiki/Ardent...,0.0,0.0,0.0,0.0,60.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,axiom_arc,3000,96.87,2906.10,https://leagueoflegends.fandom.com/wiki/Axiom_Arc,25.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,b_f_sword,1300,107.69,1399.97,https://leagueoflegends.fandom.com/wiki/B._F._...,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,bamis_cinder,1100,72.73,800.03,https://leagueoflegends.fandom.com/wiki/Bami's...,0.0,300.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [89]:
value = 'goredrinker'

items_df.loc[items_df['item_name']==value]

,item_name,cost,gold_efficiency,effective_value,item_url,ability_haste,health,magic_resistance,armor,ability_power,...,magic_penetration,critical_strike_chance,slow_resist,omnivamp,base_health_regeneration,health_on-hit,health_per_5_seconds,omnivamp_against_monsters,armor_penetration,tenacity
78,goredrinker,3300,110.76,3655.08,https://leagueoflegends.fandom.com/wiki/Goredr...,20.0,300.0,0.0,0.0,0.0,...,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0


In [94]:
master_list

[{'item_name': 'abyssal_mask',
  'cost': 2700,
  'gold_efficiency': 74.32,
  'effective_value': 2006.64,
  'item_url': 'https://leagueoflegends.fandom.com/wiki/Abyssal_Mask',
  'ability_haste': 10,
  'health': 450,
  'magic_resistance': 30},
 {'item_name': 'aegis_of_the_legion',
  'cost': 1400,
  'gold_efficiency': 100.48,
  'effective_value': 1406.72,
  'item_url': 'https://leagueoflegends.fandom.com/wiki/Aegis_of_the_Legion',
  'ability_haste': 10,
  'armor': 30,
  'magic_resistance': 30},
 {'item_name': 'aether_wisp',
  'cost': 850,
  'gold_efficiency': 76.76,
  'effective_value': 652.46,
  'item_url': 'https://leagueoflegends.fandom.com/wiki/Aether_Wisp',
  'ability_power': 30},
 {'item_name': 'amplifying_tome',
  'cost': 435,
  'gold_efficiency': 100.0,
  'effective_value': 435.0,
  'item_url': 'https://leagueoflegends.fandom.com/wiki/Amplifying_Tome',
  'ability_power': 20},
 {'item_name': 'anathemas_chains',
  'cost': 2500,
  'gold_efficiency': 90.67,
  'effective_value': 2266.7

In [95]:
import json

items_df.to_csv('scraped_data/summoners_rift_items.csv', index=False)

with open('scraped_data/summoners_rift_items.json','w') as f:
    json.dump(master_list,f)

with open('scraped_data/item_urls.json','w') as f:
    json.dump(item_url_list,f)
    
# with open('scraped_data/mongo_list.json','w') as f:
#     json.dump(mongo_list,f)